In [1]:
import math
from itertools import product

import numpy as np

from pyinverse.grid import RegularGrid
from pyinverse.axes import RegularAxes3
from pyinverse.ray3 import regular_axes2polytope, grid_uv2half_planes, beam2actor
from pyinverse.volume import volume_cal

from pyviz3d.viz import Renderer

In [2]:
#Nx = Ny = Nz = 9
Nx = 9 
Ny = 10
Nz = 11
axes3 = RegularAxes3.linspace((-1, 1, Nx), (-1, 1, Ny), (-1, 1, Nz))

In [3]:
Nu = Nv = 11

mn = (4, 5)

#phi_deg = 15
#theta_deg = 25
phi_deg = 45
theta_deg = 15

grid_uv = RegularGrid.linspace((-1, 1, Nu), (-1, 1, Nv))

In [4]:
A_mn, b_mn = grid_uv2half_planes(theta_deg, phi_deg, grid_uv, mn, degrees=True)

In [5]:
def ray_row(A_mn, b_mn, u_T, v_T, axes3):
    Nz, Ny, Nx = axes3.shape
    
    data = []
    indices = []
    
    for kji in product(range(Nx), range(Ny), range(Nz)):
        ijk = kji[::-1]
        A_ijk, b_ijk = regular_axes2polytope(axes3, ijk)

        A_lass = np.vstack((A_ijk, A_mn))
        b_lass = np.hstack((b_ijk, b_mn))
        
        vol = volume_cal(10, 3, A_lass, b_lass) / (u_T * v_T)
        if not np.allclose(vol, 0):
            data.append(vol)
            indices.append(RegularAxes3.ravel_multi_index(([ijk[2]], [ijk[1]], [ijk[0]]), axes3.shape)[0])
    
    sorted_indices, sorted_data = list(zip(*sorted(zip(indices, data), key=lambda x: x[0])))
    return sorted_data, sorted_indices

In [6]:
data, indices = ray_row(A_mn, b_mn, grid_uv.axis_x.T, grid_uv.axis_y.T, axes3)

In [7]:
def regular_axes2polytope2(axes3, i1, i2, j1, j2, k1, k2):
    """
    """
    A = [[-1,  0,  0],
         [ 1,  0,  0],
         [ 0, -1,  0],
         [ 0,  1,  0],
         [ 0,  0, -1],
         [ 0,  0,  1]]
    b = [-axes3.axis_z.borders[i1],
          axes3.axis_z.borders[i2],
         -axes3.axis_y.borders[j1],
          axes3.axis_y.borders[j2],
         -axes3.axis_x.borders[k1],
          axes3.axis_x.borders[k2]]
    return A, b

In [8]:
i1 = 0
i2 = 7

(i1, i2//2), (i2//2+1, i2)

((0, 3), (4, 7))

In [9]:
math.ceil(1/2)

1

In [10]:
def test(i1, i2):
    def test_helper(i_list, i1, i2):
        #print(i1, i2)
        assert i2 > i1
        if i2 == i1 + 1:
            i_list.append((i1, i2))
        else:
            b = i2 - i1
            c = math.ceil(b/2) + i1
            test_helper(i_list, i1, c)
            test_helper(i_list, c, i2)
        return i_list
    
    return test_helper([], i1, i2)

In [11]:
test(0, 10)

[(0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10)]

In [12]:
def test2(i1, i2, j1, j2):
    def test_helper(i_list, i1, i2, j1, j2):
        if (i2 <= i1) or (j2 <= j1):
            return
        if (i2 == i1 + 1) and (j2 == j1 + 1):
            i_list.append((i1, i2, j1, j2))
        else:
            bi = i2 - i1
            ci = math.ceil(bi/2) + i1
            
            bj = j2 - j1
            cj = math.ceil(bj/2) + j1
            
            test_helper(i_list, i1, ci, j1, cj)
            test_helper(i_list, ci, i2, j1, cj)
            test_helper(i_list, i1, ci, cj, j2)
            test_helper(i_list, ci, i2, cj, j2)
        return i_list
    
    return test_helper([], i1, i2, j1, j2)

In [13]:
sorted(test2(0, 10, 0, 3))

[(0, 1, 0, 1),
 (0, 1, 1, 2),
 (0, 1, 2, 3),
 (1, 2, 0, 1),
 (1, 2, 1, 2),
 (1, 2, 2, 3),
 (2, 3, 0, 1),
 (2, 3, 1, 2),
 (2, 3, 2, 3),
 (3, 4, 0, 1),
 (3, 4, 1, 2),
 (3, 4, 2, 3),
 (4, 5, 0, 1),
 (4, 5, 1, 2),
 (4, 5, 2, 3),
 (5, 6, 0, 1),
 (5, 6, 1, 2),
 (5, 6, 2, 3),
 (6, 7, 0, 1),
 (6, 7, 1, 2),
 (6, 7, 2, 3),
 (7, 8, 0, 1),
 (7, 8, 1, 2),
 (7, 8, 2, 3),
 (8, 9, 0, 1),
 (8, 9, 1, 2),
 (8, 9, 2, 3),
 (9, 10, 0, 1),
 (9, 10, 1, 2),
 (9, 10, 2, 3)]

In [14]:
def test3(i1, i2, j1, j2, k1, k2):
    def test_helper(i_list, i1, i2, j1, j2, k1, k2):
        if (i2 <= i1) or (j2 <= j1) or (k2 <= k1):
            return
        if (i2 == i1 + 1) and (j2 == j1 + 1) and (k2 == k1 + 1):
            i_list.append((i1, i2, j1, j2, k1, k2))
        else:
            bi = i2 - i1
            ci = math.ceil(bi/2) + i1
            
            bj = j2 - j1
            cj = math.ceil(bj/2) + j1
            
            bk = k2 - k1
            ck = math.ceil(bk/2) + k1
            
            test_helper(i_list, i1, ci, j1, cj, k1, ck)
            test_helper(i_list, ci, i2, j1, cj, k1, ck)
            test_helper(i_list, i1, ci, cj, j2, k1, ck)
            test_helper(i_list, ci, i2, cj, j2, k1, ck)
            
            test_helper(i_list, i1, ci, j1, cj, ck, k2)
            test_helper(i_list, ci, i2, j1, cj, ck, k2)
            test_helper(i_list, i1, ci, cj, j2, ck, k2)
            test_helper(i_list, ci, i2, cj, j2, ck, k2)
        return i_list
    
    return test_helper([], i1, i2, j1, j2, k1, k2)

In [15]:
len(test3(0, 10, 0, 3, 0, 5))

150

In [16]:
def ray_row2(A_mn, b_mn, u_T, v_T, axes3):
    Nz, Ny, Nx = axes3.shape
    
    data = []
    indices = []
    
    def ray_helper(data_i, indices_i, i1, i2, j1, j2, k1, k2):
        if (i2 <= i1) or (j2 <= j1) or (k2 <= k1):
            return
        
        A_ijk, b_ijk = regular_axes2polytope2(axes3, i1, i2, j1, j2, k1, k2)
        
        A_lass = np.vstack((A_ijk, A_mn))
        b_lass = np.hstack((b_ijk, b_mn))
        
        vol = volume_cal(10, 3, A_lass, b_lass) / (u_T * v_T)
        
        if np.allclose(vol, 0):
            return
        
        if (i2 == i1 + 1) and (j2 == j1 + 1) and (k2 == k1 + 1):
            data_i.append(vol)
            indices_i.append((k1, j1, i1))
            
        else:
            bi = i2 - i1
            ci = math.ceil(bi/2) + i1
            
            bj = j2 - j1
            cj = math.ceil(bj/2) + j1
            
            bk = k2 - k1
            ck = math.ceil(bk/2) + k1
            
            ray_helper(data_i, indices_i, i1, ci, j1, cj, k1, ck)
            ray_helper(data_i, indices_i, ci, i2, j1, cj, k1, ck)
            ray_helper(data_i, indices_i, i1, ci, cj, j2, k1, ck)
            ray_helper(data_i, indices_i, ci, i2, cj, j2, k1, ck)
            
            ray_helper(data_i, indices_i, i1, ci, j1, cj, ck, k2)
            ray_helper(data_i, indices_i, ci, i2, j1, cj, ck, k2)
            ray_helper(data_i, indices_i, i1, ci, cj, j2, ck, k2)
            ray_helper(data_i, indices_i, ci, i2, cj, j2, ck, k2)
        return data_i, indices_i
            
    data, ijk = ray_helper([], [], 0, Nz, 0, Ny, 0, Nx)
    flat_indices = RegularAxes3.ravel_multi_index(list(zip(*ijk)), axes3.shape)
    sorted_indices, sorted_data = list(zip(*sorted(zip(flat_indices, data), key=lambda x: x[0])))
    return sorted_data, sorted_indices

In [17]:
data2, indices2 = ray_row2(A_mn, b_mn, grid_uv.axis_x.T, grid_uv.axis_y.T, axes3)

In [18]:
len(data), len(data2)

(64, 64)

In [19]:
np.allclose(data, data2)

True

In [20]:
indices == indices2

True

In [21]:
e_min_max = (-1.2*np.sqrt(2), 1.2*np.sqrt(2))
beam_actor = beam2actor(grid_uv, mn, e_min_max, theta_deg, phi_deg, deg=True)

In [22]:
X = np.zeros(axes3.shape)
X.flat[np.array(indices)] = data

In [23]:
ijk_set = set(zip(*RegularAxes3.unravel_indices(indices, axes3.shape)))

for ijk in product(range(Nx), range(Ny), range(Nz)):
    if ijk not in ijk_set:
        k, j, i = ijk
        X[i, j, k] = np.nan

In [24]:
X_actor = axes3.actor(X, blank_nan=True)

In [25]:
ren = Renderer()
ren.add_actor(beam_actor)
ren.add_actor(X_actor)
ren.axes_on((-1, 1, -1, 1, -1, 1))
ren.reset_camera()
ren.start()